In [36]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import xgboost
from sklearn.utils import shuffle
import operator

Load the models

In [18]:
clickingModel = xgboost.Booster({'nthread': 12})  # init model
clickingModel.load_model('xgboostBinary26April.model')  # load data

bookingModel = xgboost.Booster({'nthread': 12})  # init model
bookingModel.load_model('bookingModel.model')  # load data


In [22]:
feature_names_list = ['visitor_hist_starrating', 'visitor_hist_adr_usd',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'comp1_rate', 'comp1_inv',
                 'prop_starrating_mean',
       'prop_review_score_mean', 'prop_brand_bool_mean',
       'prop_location_score1_mean', 'prop_location_score2_mean',
       'prop_log_historical_price_mean']
featTypes =['int',
 'int',
 'int',
 'float',
 'float',
 'int',
 'int',
 'int',
 'float',
 'int',
 'float',
 'float',
 'float',
 'float',
 'int',
 'int',
 'int',
 'int',
 'int',
 'int',
 'int',
 'int',
 'float',
 'float',
 'int',
 'float',
 'float']

In [39]:
clickingModel.feature_names = feature_names_list
clickingModel.feature_types = featTypes
bookingModel.feature_names = feature_names_list
bookingModel.feature_types = featTypes
scoresClick = clickingModel.get_score(importance_type='weight')
print(sorted(scoresClick.items(), key=operator.itemgetter(1)))
print()
scoresBook =  bookingModel.get_score(importance_type='weight')
print(sorted(scoresBook.items(), key=operator.itemgetter(1)))

[('comp1_inv', 19), ('srch_saturday_night_bool', 23), ('comp1_rate', 25), ('prop_brand_bool', 95), ('srch_room_count', 98), ('visitor_hist_starrating', 102), ('promotion_flag', 106), ('srch_children_count', 117), ('srch_length_of_stay', 117), ('visitor_location_country_id', 131), ('site_id', 133), ('srch_query_affinity_score', 140), ('random_bool', 147), ('visitor_hist_adr_usd', 160), ('srch_adults_count', 171), ('srch_id', 192), ('srch_booking_window', 266), ('prop_country_id', 303), ('prop_starrating', 348), ('prop_review_score', 351), ('prop_id', 355), ('srch_destination_id', 386), ('orig_destination_distance', 390), ('prop_log_historical_price', 454), ('prop_location_score1', 657), ('prop_location_score2', 954), ('price_usd', 984)]

[('comp1_inv', 3), ('visitor_location_country_id', 6), ('site_id', 6), ('comp1_rate', 7), ('prop_id', 12), ('prop_log_historical_price', 15), ('srch_id', 17), ('srch_destination_id', 18), ('srch_children_count', 20), ('prop_country_id', 21), ('random_bo

Load the data

In [3]:
OnlyGoodCols = ['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv']

In [13]:
df = pd.read_csv('../data/train.csv', delimiter=",", usecols = ['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv', 'click_bool', 'gross_bookings_usd',
       'booking_bool'])

xgmat_submission = xgboost.DMatrix(df[OnlyGoodCols])

In [4]:
df = pd.read_csv('../data/test.csv')
xgmat_submission = xgboost.DMatrix(df[OnlyGoodCols])

In [5]:
y_pred_clicking = clickingModel.predict(xgmat_submission)
y_pred_booking = bookingModel.predict(xgmat_submission)
y_pred = y_pred_booking+y_pred_clicking

In [6]:
df['y_pred'] = y_pred

In [11]:
df = df.sort_values(['srch_id','y_pred'],ascending=[True, False])


In [14]:
len(df)

4958347

In [ ]:
df[['srch_id','prop_id']].to_csv("submissionRens29April.csv",index=False)

In [ ]:
df['order']=df.groupby('srch_id').cumcount().add(1)